In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

!pip install contractions
import contractions

     |████████████████████████████████| 321 kB 8.0 MB/s 
     |████████████████████████████████| 284 kB 49.6 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85454 sha256=834dc15ff3a04448b8a346eef21b26656f66740f68d213f6a9b623eb1fa40094
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving vaccination_all_tweets.csv to vaccination_all_tweets.csv


In [ ]:
import io
df_train= pd.read_csv(io.BytesIO(uploaded["vaccination_all_tweets.csv"]),encoding="ISO-8859-1")

In [ ]:
df_train

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1337858199140118533,elið±ð¹ðªðºð,Your Bed,"heil, hydra ðâº",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223283,1457203761676312585,VaxBLR,"Bengaluru, India",Hourly updates on FREE and PAID 18+ and 45+ va...,2021-06-21 08:44:34,31,0,0,False,2021-11-07 04:30:24,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,"['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHI...",VaxBlr,0,1,False
223284,1457188663230492674,VaxBLR,"Bengaluru, India",Hourly updates on FREE and PAID 18+ and 45+ va...,2021-06-21 08:44:34,31,0,0,False,2021-11-07 03:30:25,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,"['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHI...",VaxBlr,0,1,False
223285,1457173563215994883,VaxBLR,"Bengaluru, India",Hourly updates on FREE and PAID 18+ and 45+ va...,2021-06-21 08:44:34,31,0,0,False,2021-11-07 02:30:25,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,"['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHI...",VaxBlr,0,1,False
223286,1457158462178140170,VaxBLR,"Bengaluru, India",Hourly updates on FREE and PAID 18+ and 45+ va...,2021-06-21 08:44:34,31,0,0,False,2021-11-07 01:30:24,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,"['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHI...",VaxBlr,0,1,False


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223288 entries, 0 to 223287
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                223288 non-null  int64 
 1   user_name         223286 non-null  object
 2   user_location     157954 non-null  object
 3   user_description  207000 non-null  object
 4   user_created      223288 non-null  object
 5   user_followers    223288 non-null  int64 
 6   user_friends      223288 non-null  int64 
 7   user_favourites   223288 non-null  int64 
 8   user_verified     223288 non-null  bool  
 9   date              223288 non-null  object
 10  text              223288 non-null  object
 11  hashtags          174490 non-null  object
 12  source            223169 non-null  object
 13  retweets          223288 non-null  int64 
 14  favorites         223288 non-null  int64 
 15  is_retweet        223288 non-null  bool  
dtypes: bool(2), int64(6), object(8)
memory

In [ ]:
df_train.columns

Index(['id', 'user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'retweets', 'favorites',
       'is_retweet'],
      dtype='object')

In [ ]:
df_train["len"]= df_train["text"].apply(lambda x : len(x))

In [ ]:
df_train=df_train[["text","len"]]

In [ ]:
df_train

,text,len
0,Same folks said daikon paste could treat a cyt...,97
1,While the world has been on the wrong side of ...,142
2,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,142
3,"Facts are immutable, Senator, even when you're...",142
4,Explain to me again why we need a vaccine @Bor...,137
...,...,...
223283,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,142
223284,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,142
223285,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,142
223286,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,142


In [ ]:
#delete duplicates
df_train.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
import re
df_train["text"]=df_train["text"].apply(lambda x: x.lower())
df_train["text"]=df_train["text"].apply(lambda x: contractions.fix(x))
df_train["text"]=df_train["text"].apply(lambda x: re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)',' ', x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
import nltk
nltk.download('wordnet')

def stemming(corpus):
    
    corpus_=""
    
    ps = nltk.stem.porter.PorterStemmer()
    for mot in corpus.split():
        corpus_=corpus_ + " " + ps.stem(mot) + " "
        
    return corpus_

def lemmatisation(corpus):
    corpus_=""
    lem = nltk.stem.wordnet.WordNetLemmatizer()
    for mot in corpus.split():
        corpus_=corpus_ + " " + lem.lemmatize(mot) + " "
    return corpus_

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
import nltk 
from nltk.corpus import stopwords

def remove_stopwords(corpus):
    corpus_=""
    for mot in corpus.split():
        if mot not in stopwords.words('english'):
            corpus_=corpus_ + " " + mot
        else:
            corpus_=corpus_ + " "
    return corpus_

In [ ]:
df_train["text"]=df_train["text"].apply(lambda x: stemming(x))
df_train["text"]=df_train["text"].apply(lambda x: lemmatisation(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
import nltk
nltk.download('stopwords')
  
df_train["text"]=df_train["text"].apply(lambda x: remove_stopwords(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [18]:

from gensim.utils import simple_preprocess

tokens=df_train["text"].apply(lambda x : simple_preprocess(x))

In [19]:
from gensim import corpora
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(text) for text in tokens]

In [20]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=8)
for topic in topics:
    print(topic)

(0, '0.056*"vaccin" + 0.045*"covid" + 0.029*"approv" + 0.025*"use" + 0.024*"india" + 0.019*"emerg" + 0.018*"ha" + 0.016*"sinovac"')
(1, '0.047*"moderna" + 0.030*"bharatbiotech" + 0.029*"vaccin" + 0.023*"got" + 0.016*"get" + 0.014*"shot" + 0.014*"sinopharm" + 0.012*"today"')
(2, '0.038*"moderna" + 0.032*"vaccin" + 0.030*"who" + 0.023*"covid" + 0.020*"approv" + 0.019*"pfizer" + 0.018*"booster" + 0.012*"thi"')
(3, '0.088*"age" + 0.070*"pincode" + 0.064*"date" + 0.062*"fees" + 0.060*"ages" + 0.031*"bbmp" + 0.028*"oct" + 0.027*"sep"')
(4, '0.166*"covaxin" + 0.099*"dose" + 0.096*"slot" + 0.043*"covidvaccin" + 0.043*"free" + 0.040*"covishield" + 0.039*"avail" + 0.036*"bengaluru"')
